In [1]:
import cv2
import numpy as np
import tensorflow as tf


In [2]:
model = tf.keras.models.load_model(
    "C:/Users/ismai/HandGestureRecognition/models/webcam_gesture_cnn.keras"
)

print("✅ Modèle chargé")


✅ Modèle chargé


In [2]:
import cv2
import numpy as np
import tensorflow as tf

# 1. Charger le modèle (Utilise le chemin exact de ton fichier .keras)
model_path = "C:/Users/ismai/HandGestureRecognition/models/webcam_gesture_cnn.keras"
model = tf.keras.models.load_model(model_path)

# 2. Définir les classes (L'ordre doit correspondre à tes dossiers)
classes = ["fist", "palm", "victory"]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Miroir pour plus de confort
    frame = cv2.flip(frame, 1)
    
    h, w, _ = frame.shape
    size = 300
    cx, cy = w // 2, h // 2
    
    # Zone d'intérêt (ROI)
    x1, y1, x2, y2 = cx - size//2, cy - size//2, cx + size//2, cy + size//2
    roi = frame[y1:y2, x1:x2]

    # --- ÉTAPE CRUCIALE : PRÉTRAITEMENT BINAIRE ---
    # Pour que ça marche, la webcam doit "voir" comme pendant l'entraînement
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 2)
    # Seuillage d'Otsu pour isoler la main en blanc sur fond noir
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Préparation pour le modèle
    img_ready = cv2.resize(thresh, (64, 64))
    img_ready = img_ready.reshape(1, 64, 64, 1) / 255.0

    # --- PRÉDICTION ---
    pred = model.predict(img_ready, verbose=0)
    confidence = np.max(pred)
    class_id = np.argmax(pred)

    # --- AFFICHAGE ---
    if confidence < 0.70:
        label = "Inconnu"
        color = (0, 0, 255) # Rouge
    else:
        label = f"{classes[class_id]} ({confidence*100:.1f}%)"
        color = (0, 255, 0) # Vert

    # Dessiner l'interface
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Afficher les deux fenêtres
    cv2.imshow("Detection de Gestes", frame)
    cv2.imshow("Vue du Modele (Threshold)", thresh)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()